# Loan Underwriter Flow (FloTorch Variant)

This simple agentic flow evaluates loan applications using LLM-powered agents provided by FloTorch.

There are 3 agents operating in this workflow:

1. Loan Officer - Summarizes the application and highlights significant information.
2. Credit Analyst - Rates applicant creditworthiness (Low/Medium/High).
3. Risk Manager - Makes the final decision (Approved/Denied) with an explanation.

### Prerequisites

1. [FloTorch](https://www.flotorch.ai/) account
2. FloTorch API key
3. Configured agents on the FloTorch platform

Use the [Getting Started](https://docs.google.com/document/d/1NZ1ZY9uCDYzqMOk6WG3Fb4aaWq2y_ArxMKsifUiUTgg/edit?usp=sharing) guide.

## Install packages
- FloTorch LangGraph SDK
- LangGraph

In [ ]:
# %pip install --pre flotorch[autogen] # beta version
%pip install flotorch[langgraph] # production version
%pip install langgraph

## Configure FloTorch
Set your API key.

In [ ]:
try: # if in Google Colab
    import google.colab
    FLOTORCH_API_KEY = userdata.get("FLOTORCH_API_KEY")
    
except ImportError: # if running locally
    import os
    from dotenv import load_dotenv
    load_dotenv()  # reads .env in current directory
    FLOTORCH_API_KEY = os.getenv("FLOTORCH_API_KEY")

if not FLOTORCH_API_KEY:
    raise ValueError("FLOTORCH_API_KEY not found. Set it in .env or enter it in Colab Secrets.")

FLOTORCH_GATEWAY_BASE_URL = "https://gateway.flotorch.cloud"

## Connect to agents

Find the names of the agents you created under the **Agents** tab in the FloTorch console.

In [ ]:
import nest_asyncio
nest_asyncio.apply() # run async code in Jupyter notebooks
from flotorch.langgraph.agent import FlotorchLangGraphAgent

AGENTS = {
    "loan_parser": "loan-parser",
    "credit_analyzer": "credit-risk-analyzer",
    "risk_assessor": "risk-assessor"
}

loan_parser_client = FlotorchLangGraphAgent(
    agent_name=AGENTS['loan_parser'],
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_GATEWAY_BASE_URL
)

credit_analyzer_client = FlotorchLangGraphAgent(
    agent_name=AGENTS['credit_analyzer'],
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_GATEWAY_BASE_URL
)

risk_assessor_client = FlotorchLangGraphAgent(
    agent_name=AGENTS['risk_assessor'],
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_GATEWAY_BASE_URL
)

## Build agent flow

Define LangGraph wrapper function to input FlotorchLangGraphAgent as a LangGraph node.

In [ ]:
import asyncio
from langchain_core.runnables import RunnableLambda

def get_agent_callable(agent_client, input_key="messages"):
    """
    Wrap a FlotorchLangGraphAgent so it can be used as a StateGraph node.

    Args:
        agent: FlotorchLangGraphAgent instance
        input_key: the attribute in the state object that contains the input text
    Returns:
        RunnableLambda: LangGraph-compatible node
    """
    agent = agent_client.get_agent()

    def node_callable(state, context=None):
        # Extract input from the state object
        if isinstance(state, dict):
            messages = state.get(input_key, "")
        else:
            messages = getattr(state, input_key, "")

        # Call the agent safely inside the running event loop
        loop = asyncio.get_event_loop()
        if loop.is_running():
            future = asyncio.ensure_future(
                agent.ainvoke({"messages": messages})
            )
            loop.run_until_complete(future)
            result = future.result()
        else:
            result = loop.run_until_complete(
                agent.ainvoke({"messages": messages})
            )

        # Return the content of the last message
        content = result["messages"][-1].content
        state = {
            "messages": content
        }
        return state

    return RunnableLambda(node_callable)

Build the LangGraph StateGraph to orchestrate the agent workflow.

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph

class LoanState(TypedDict):
    messages: str
    

graph = StateGraph(LoanState)

graph.add_node("LoanParser", get_agent_callable(loan_parser_client))
graph.add_node("CreditAnalyzer", get_agent_callable(credit_analyzer_client))
graph.add_node("RiskAssessor", get_agent_callable(risk_assessor_client))

graph.set_entry_point("LoanParser")
graph.add_edge("LoanParser", "CreditAnalyzer")
graph.add_edge("CreditAnalyzer", "RiskAssessor")
graph.set_finish_point("RiskAssessor")

loan_underwriter = graph.compile()

## Create loan application

Fill in necessary fields as you please.

In [ ]:
loan_application = {
    "name": "Jane Doe",
    "age": 35,
    "income": 200000,
    "loan_amount": 35000,
    "credit_score": 720,
    "existing_liabilities": 150000,
    "purpose": "Medical Emergency"
}

## Run flow

Invoke the graph with the input.

In [ ]:
import json
from IPython.display import display, Markdown

loan_state: LoanState = {
    "messages": str(loan_application)
} 

result = loan_underwriter.invoke(loan_state)
output = result["messages"]
data = json.loads(output)
loan_decision = data["loan_decision"]
risk_assessment = data["risk_assessment"]

display(Markdown(f"Decision: {loan_decision}"))
display(Markdown(f"Explanation: {risk_assessment}"))

This is a version of the flow without the LangGraph package (no StateGraph or LangGraph node wrapper function).

In [ ]:
loan_application_input = str(loan_application)

loan_parser_agent = loan_parser_client.get_agent()
loan_parser_response = loan_parser_agent.invoke({"messages": loan_application_input})
loan_parser_output = loan_parser_response['messages'][-1].content

credit_analyzer_agent = credit_analyzer_client.get_agent()
credit_analyzer_response = credit_analyzer_agent.invoke({"messages": loan_parser_output})
credit_analyzer_output = credit_analyzer_response['messages'][-1].content

risk_assessor_agent = risk_assessor_client.get_agent()
risk_assessor_response = risk_assessor_agent.invoke({"messages": credit_analyzer_output})
risk_assessor_output = risk_assessor_response['messages'][-1].content

print(f"final output:::::::::::::\n{risk_assessor_output}")